#### Imports

In [1]:
import cv2, os
from keras.applications.mobilenet import MobileNet
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras.metrics import Precision, Recall, F1Score
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

#### Variables

In [2]:
data_filepath = 'data\data'
categories = ['with_mask', 'without_mask']
categories_mask = {'with_mask': 1, 'without_mask': 0}
df = pd.DataFrame()
model_filepath = "model.h5"

#### Data Loading - Organizing

##### Downloading

In [ ]:
# !kaggle datasets download -d omkargurav/face-mask-dataset

In [6]:
# with zipfile.ZipFile("face-mask-dataset.zip", "r") as file:
#     file.extractall("data")

##### Creating dataframe

In [ ]:
# reading each image, reshaping it, converting to numpy array and saving the array along with the image category
dataset = {
  'image': [],
  'category': []
}

for  category in categories:
  dirpath = os.path.join(data_filepath, category)
  for img_filename in os.listdir(dirpath):
    img_path = os.path.join(dirpath, img_filename)

    img = cv2.imread(img_path)
    img = cv2.resize(img, dsize=(224,224))
    img = np.array(img)
    # print(img.shape)
    
    dataset['image'].append(img)
    dataset['category'].append(category)

print(f"{img_filename} \
      \n{dataset['image'][0]} \
      \n{dataset['category'][0]}")

In [40]:
df = pd.DataFrame(dataset, columns=['image', 'category'])
df.head(1)

,image,category
0,"[[[35, 63, 57], [28, 55, 46], [32, 55, 45], [3...",with_mask


In [41]:
df.shape

(7553, 2)

In [42]:
df = df.sample(frac=1).reset_index(drop=True)
df.head(3)

,image,category
0,"[[[197, 202, 201], [197, 202, 201], [197, 202,...",without_mask
1,"[[[208, 194, 230], [205, 192, 228], [201, 188,...",without_mask
2,"[[[42, 53, 81], [41, 52, 80], [39, 50, 78], [3...",with_mask


In [45]:
df['category'] = df['category'].map(categories_mask)
df.head(1)

,image,category
0,"[[[197, 202, 201], [197, 202, 201], [197, 202,...",0


##### EDA

In [46]:
df.shape

(7553, 2)

In [48]:
print("Images with mask")
df[df['category']==1].shape[0]

Images with mask


3725

In [49]:
print("Images without mask")
df[df['category']==0].shape[0]

Images without mask


3828

#### Creating Neural Network

##### Builing Architecture

In [73]:
# mnet = MobileNet((224,224,3), include_top=False)

17225924/17225924 [==============================] - 37s 2us/step


In [74]:
mnet.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32

In [97]:
model = Sequential()
for layer in mnet.layers:
    layer.trainable = False
    model.add(layer)

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32)      128       
 zation)                                                         
                                                      

In [98]:
model.add(GlobalAveragePooling2D()) # using globalaveragepoolinglayer instead of flatten
model.add(Dense(1, 'sigmoid'))

In [82]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32)      128       
 zation)                                                         
                                                      

##### Training

In [101]:
# Convert DataFrame to tf.data.Dataset
def dataframe_to_dataset(dataframe):
    images = np.stack(dataframe['image'].values)
    labels = dataframe['category'].values.astype('float32')
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    return dataset

# Create a tf.data.Dataset
dataset = dataframe_to_dataset(df)

# Shuffle and batch the dataset
batch_size = 32
dataset = dataset.shuffle(buffer_size=len(df)).batch(batch_size)

# Prefetching for performance
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [102]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall(), F1Score()])
model.fit(dataset, epochs=5, verbose=1, batch_size=20)

Epoch 1/5
237/237 [==============================] - 144s 591ms/step - loss: 0.4266 - accuracy: 0.8121 - precision_2: 0.8121 - recall_2: 0.8054 - f1_score: 0.6606
Epoch 2/5
237/237 [==============================] - 141s 594ms/step - loss: 0.2726 - accuracy: 0.8992 - precision_2: 0.8999 - recall_2: 0.8953 - f1_score: 0.6606
Epoch 3/5
237/237 [==============================] - 138s 584ms/step - loss: 0.2368 - accuracy: 0.9126 - precision_2: 0.9156 - recall_2: 0.9063 - f1_score: 0.6606
Epoch 4/5
237/237 [==============================] - 138s 584ms/step - loss: 0.2176 - accuracy: 0.9166 - precision_2: 0.9186 - recall_2: 0.9117 - f1_score: 0.6606
Epoch 5/5
237/237 [==============================] - 154s 650ms/step - loss: 0.2049 - accuracy: 0.9269 - precision_2: 0.9324 - recall_2: 0.9184 - f1_score: 0.6606


In [103]:
model.save("model.h5")

c:\Users\Dmm\anaconda3\envs\data-science\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [104]:
model.save("model.keras")

#### App Building

In [3]:
import streamlit as st

def predict_class(captured_image):
    encoded_image = encode_cvimage(captured_image)
    model = load_model()
    prediction = model.predict(encoded_image)
    prediction = map_probability(prediction[0])
    return prediction

def encode_cvimage(cv_image):
    image = cv2.resize(cv_image, dsize=(224,224))
    image = np.array(image)
    image = np.reshape(image, (1,224,224,3))
    return image

@st.cache_resource
def load_model():
    from keras.models import load_model
    if os.path.exists(model_filepath):
        return load_model(model_filepath)
    else:
        raise Exception(f"Model path doesn't exist: {model_filepath}")
    
def map_probability(value):
    return "Without mask" if value < 0.5 else "With mask"

In [133]:
# making prediction on single image
print(predict_class(cv2.imread('image.PNG')))

1/1 [==============================] - 0s 497ms/step
Without mask


In [4]:
# making prediction on the live input
camera = cv2.VideoCapture(0)

while True:
    success, image = camera.read()
    cv2.imshow("Camera", image)
    predicted_class = predict_class(image)
    print(predict_class)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()

2024-07-28 15:28:04.719 
  command:

    streamlit run c:\Users\Dmm\anaconda3\envs\data-science\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]




1/1 [==============================] - 1s 1s/step
<function predict_class at 0x0000013015615510>
1/1 [==============================] - 1s 532ms/step
<function predict_class at 0x0000013015615510>
1/1 [==============================] - 1s 503ms/step
<function predict_class at 0x0000013015615510>
1/1 [==============================] - 1s 531ms/step
<function predict_class at 0x0000013015615510>
1/1 [==============================] - 1s 516ms/step
<function predict_class at 0x0000013015615510>
1/1 [==============================] - 1s 522ms/step
<function predict_class at 0x0000013015615510>
1/1 [==============================] - 1s 697ms/step
<function predict_class at 0x0000013015615510>
1/1 [==============================] - 1s 575ms/step
<function predict_class at 0x0000013015615510>
1/1 [==============================] - 1s 512ms/step
<function predict_class at 0x0000013015615510>
1/1 [==============================] - 1s 561ms/step
<function predict_class at 0x0000013015615510>
1